**DEEP SEEK LLM Finetuning**

In [ ]:
!pip install transformers datasets accelerate peft


In [ ]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

In [ ]:
data = [
    {"text": "Patient: I have a headache.\nDoctor: Drink warm ginger water and take some rest."},
    {"text": "Patient: My stomach is upset.\nDoctor: Avoid heavy spicy food and have a light khichdi."},
    {"text": "Patient: I feel dizzy.\nDoctor: Sit down, drink coconut water, and rest for a while."},
    {"text": "Patient: I have a sore throat.\nDoctor: Gargle with warm salt water and drink turmeric milk."},
    {"text": "Patient: I have a fever.\nDoctor: Take paracetamol, drink plenty of fluids, and rest."},
    {"text": "Patient: I feel tired all the time.\nDoctor: Eat fresh fruits like papaya and bananas, and get 7–8 hours of sleep."},
    {"text": "Patient: I have a cough.\nDoctor: Drink warm herbal tea with tulsi and honey."},
    {"text": "Patient: I have stomach cramps.\nDoctor: Apply a warm compress and avoid heavy meals like fried samosas."},
    {"text": "Patient: My back hurts.\nDoctor: Do light stretches, take a hot water bottle, and sleep on a firm mattress."},
    {"text": "Patient: I feel anxious.\nDoctor: Practice yoga breathing exercises and meditate for 10 minutes daily."}
]



In [ ]:
dataset = Dataset.from_list(data)

In [ ]:
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",       # automatically place layers on GPU
    torch_dtype=torch.float16
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r=8,                    # rank of LoRA matrices
    lora_alpha=32,          # scaling factor
    target_modules=["q_proj", "v_proj"],  # modules to apply LoRA
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        max_length=64,
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./fast_lora_finetune",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    max_steps=10,
    logging_steps=1,
    fp16=True,          # mixed precision for VRAM saving
    save_strategy="no"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kalyansundar (kalyansundar-governmentjobs-com) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,17.897400
2,18.605200
3,18.368000
4,16.760400
5,17.307000
6,14.408800
7,19.215600
8,18.745800
9,18.267500
10,17.894800


TrainOutput(global_step=10, training_loss=17.74703884124756, metrics={'train_runtime': 211.3764, 'train_samples_per_second': 0.047, 'train_steps_per_second': 0.047, 'total_flos': 4922820526080.0, 'train_loss': 17.74703884124756, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./fast_lora_finetuned")
tokenizer.save_pretrained("./fast_lora_finetuned")

('./fast_lora_finetuned/tokenizer_config.json',
 './fast_lora_finetuned/special_tokens_map.json',
 './fast_lora_finetuned/chat_template.jinja',
 './fast_lora_finetuned/tokenizer.json')

In [ ]:
def generate_response(prompt, max_length=150):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = "give python script 2 swap 2 numbers"
print("Chatbot response:", generate_response(prompt))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Chatbot response: give python script 2 swap 2 numbers


def swap_numbers(a, b):
    return b, a

a = int(input("Enter first number: "))
b = int(input("Enter second number: "))

a, b = swap_numbers(a, b)

print("After swapping: a =", a, "and b =", b)

# run the script
# python script.py
# Enter first number: 5
# Enter second number: 10
# After swapping: a = 10 and b = 5

# if you want to use the swap_numbers function in another script
# you can import it from the script
